In [2]:
using Distributed

using LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, Tables, TableOperations, StatsBase, FreqTables

In [3]:
function rlassoIVselectX(x, y, z, d, post::Bool = true)
    
    n = size(y, 1)
    numIV = size(z, 2)
    Z = hcat(x, z, makeunique = true)
    lasso_d_x = rlasso(rlasso_arg(x = x, y = d, post = post))
    Dr = lasso_d_x["residuals"]
    lasso_y_x = rlasso(rlasso_arg(x = x, y = y, post = post))
    Yr = lasso_y_x["residuals"]
    Zr = zeros(n, numIV)
    for i in 1:numIV
        lasso_z_x = rlasso(rlasso_arg(x = x, y = DataFrame(z = z[:, i]), post = post))
        Zr[:, i] = lasso_z_x["residuals"]
    end
    
    result = tsls(Dr, Yr, Zr, nothing, false)
    se = result["se"]
    vcov = result["vcov"]
    coef = result["coef"]
    # coefnames = result["coefnames"]
    res = Dict("coefnames" => coefnames, "coefficients" => coef, "vcov" => vcov, "se" => se)
    
    return res
end

rlassoIVselectX (generic function with 2 methods)